In [2]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
#imports
import numpy as np
from qiskit import QuantumCircuit
# from qiskit.providers.aer import QasmSimulator

# from qiskit.visualization import plot_histogram, plot_bloch_vector
from typing import Tuple, Any
import math
import cmath
     

In [4]:
def get_gray_code(n:int)->np.array:
    
    gray_list = []
  
    for i in range(1 << n):
        gray_binary = bin(i ^ (i >> 1))[2::]
        gray_binary = gray_binary.zfill(n)
        gray_list.append(gray_binary)

    return np.asarray(gray_list)

# unmute for testing 
get_gray_code(3)

    
    

array(['000', '001', '011', '010', '110', '111', '101', '100'],
      dtype='<U3')

In [5]:
def get_phases(state: np.array) -> np.array:
 

    omega_phases = np.empty(len(state), dtype=float)

    for st in range(len(state)):
        omega_phases[st] = cmath.phase(state[st])

    return omega_phases
  
# unmute for testing

#get_phases(state)

In [6]:
def g(i) :  # binary grey code return functions
    return i ^ (i >> 1)
def get_m_matrix_entry(i:int, j:int, k:int)-> int:
 

    bitwise_dot_product = bin(j&g(i)).count('1')
    m_ij = (2 ** (-k)) * (-1) ** bitwise_dot_product
  
    return m_ij

# for testing
get_m_matrix_entry(2, 2, 2)

-0.25

In [7]:
def alpha_y(state:np.array, n:int, k:int) -> float:
  """
  Calculates the agles alpha_i sunbited to the R_y gate.
  Args:
    state: (array-like), the state of the input vector to be transformed.
    n: (int), the number of qubits in circuit.
    k: (int), the number of controll nodes.
  Returns:
    alpha :(float), the value of rotation angle (r_y quatum gate).
  """
  
  magnitude = [abs(st) for st in state]
  magnitude = np.asarray(magnitude)

  denominator = 0
  numerator = 0
  for j in range(2 ** (n - k)):
    # construct denominator 
    for l in range(2 ** k):
      index = (j) * (2 ** k) + l
      denominator += (magnitude[index]) ** 2 
  
    # construct numerator
    for l in range(2 ** (k - 1)):
      index = (2 * (j+1) - 1) * (2 ** (k - 1)) + l 
      numerator += (magnitude[index]) ** 2 

  return 2 * math.asin(math.sqrt(numerator / denominator))

# for testing 
# state = np.array([2, 2j, 6, 6j]) 
# alpha_y(state, 2, 2)

In [51]:
def alpha_z(omega_phases:np.array, n:int, k:int) -> float:
  """
  Calcualtes angles alpha_i submitted tot he rotation gate R_z.
  Args:
    omega_phases: (np.array), phases of the input vector.
    n: (int), the number of qubits in circuit.
    k: (int), the number of the control nodes. 
  Returns:
    alpha_z: (float), the value of rotation angle (r_z quatum gate).
  """

  numerator = 0
  for j in range(1, 2 ** (n - k) + 1):
    # construct denominator 
    for l in range(1, 2 ** (k-1) + 1):
      expression1 = (2 * (j) - 1) * (2 ** (k - 1)) + l - 1
      expression2 = (2 * (j) - 2) * (2 ** (k - 1)) + l - 1
      numerator += (omega_phases[expression1] - omega_phases[expression2])

  return numerator / 2 ** (k - 1)

# for testing
# phases = get_phases(state)
# alpha_z(phases, 2,1)
     

In [52]:
def apply_gate_sequence(q_circuit, gate, angle:np.array, k:int, target:int)->None:
  """
  Applies the sequence of quantum gates based the Gray code representation for 
  the controll nodes.

  Args:
    q_circuit: an object of the class 'qiskit.circuit.quantumcircuit.QuantumCircuit. 
              A gate is to be applied on the q_circuit.
    gate: a method of the  class 'qiskit.circuit.quantumcircuit.QuantumCircuit
           a qiskit quantum gate object to be applied on the quantum circuit.
    angle: (np.array) angle to be transformed using the M_ij matrix.
    k: (int), a number of controll qubits.
    target: (int), a node of the target qubit.
  Returns:
    None.
  """

  # compute theta angles as discribed in eq. 3
  size = len(angle)
  
  matrix_M = np.zeros(shape=(size, size))

  for i in range(size):
    for j in range(size):
      matrix_M[i][j] = get_m_matrix_entry(i, j, k)

  theta_angles = np.dot(matrix_M, np.transpose(angle))

  theta_angles = np.transpose(theta_angles)
  
  gray_code = get_gray_code(k)
  
  control_gates = []
  for i in range(len(gray_code)):
    # representing Gray code binaries as regular binaries through XOR operator
    control = int(gray_code[i], 2) ^ int(gray_code[(i+1) % len(gray_code)], 2)
    
    control = int(np.log2(control))
    
    control_gates.append(control)
  

  if len(control_gates) == 2:
    q_circuit.cx(0, 1)
    if gate == 'ry':
        q_circuit.ry(theta_angles[0], 1)
    elif gate == 'rz':
        q_circuit.rz(theta_angles[0], 1)

  for i in range(len(control_gates)-2, 0, -1):
    # aply cx gates
    j = control_gates[i]
    while j != target:
      q_circuit.cx(j, j+1)
      j += 1

    # apply rotation gates
    if gate == 'ry':
        q_circuit.ry(theta_angles[i], target)
    elif gate == 'rz':
        q_circuit.rz(theta_angles[i], target)
    

In [53]:
def state_prep(input_vector):

  # normalize state
  state = input_vector / np.linalg.norm(input_vector)
  
  # get number of qubits for circuit based on the lenght of the state
  num_qubits = int(math.log2(len(state)))
  
  qc = QuantumCircuit(num_qubits)

  phases = get_phases(state)
  angles_alpha_z = [alpha_z(phases, num_qubits, k) for k in range(1, num_qubits+1)]
  angles_alpha_y = [alpha_y(state, num_qubits, k) for k in range(1, num_qubits+1)]

  # sequence of control rz gates
  for control in range(num_qubits-1,  0, -1):
      apply_gate_sequence(qc, 'rz', angles_alpha_z, control, control)
  qc.rz(angles_alpha_z[num_qubits-1], num_qubits-1)

  # sequence of control ry gates
  for control in range(num_qubits-1,  0, -1):
      apply_gate_sequence(qc, 'ry', angles_alpha_y, control, control)
  qc.ry(angles_alpha_y[num_qubits-1], num_qubits-1)

  # inverse the circuit to prepare the second circuit.
  qc_inv = qc.inverse()
  qc = qc.compose(qc_inv)

  return state, qc.draw()
     

In [54]:
input_states =[[1-3j, 5+10j], [1-2j, 3+1j, 4j, 5], [1-3j, 5+10j, 1-2j, 3+1j, 7j, 3, -3-6j, -10+1j]]

for state in input_states:
  state_prepared, circuit_representation = state_prep(np.asarray(state))
  print("Initial state is \n {}". format(state))
  print("Mottonen state prepared is \n {} .". format(state_prepared))
  print("Quantum circuit: \n", circuit_representation)
  print("=========================")

Initial state is 
 [(1-3j), (5+10j)]
Mottonen state prepared is 
 [0.0860663 -0.25819889j 0.43033148+0.86066297j] .
Quantum circuit: 
 Figure(314.126x84.28)
Initial state is 
 [(1-2j), (3+1j), 4j, 5]
Mottonen state prepared is 
 [0.13363062-0.26726124j 0.40089186+0.13363062j 0.        +0.53452248j
 0.6681531 +0.j        ] .
Quantum circuit: 
 Figure(808.852x144.48)
Initial state is 
 [(1-3j), (5+10j), (1-2j), (3+1j), 7j, 3, (-3-6j), (-10+1j)]
Mottonen state prepared is 
 [ 0.0531494-0.1594482j  0.265747 +0.531494j   0.0531494-0.1062988j
  0.1594482+0.0531494j  0.       +0.3720458j  0.1594482+0.j
 -0.1594482-0.3188964j -0.531494 +0.0531494j] .
Quantum circuit: 
 Figure(1531.4x204.68)
